In [1]:

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
#고급
import tensorflow as tf
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None, 784])  # 입력될 이미지
y_ = tf.placeholder(tf.float32, shape=[None, 10])  # 계산해서 나온결과 ,정답을 저장하는 텐서 (배열)

W = tf.Variable(tf.zeros([784,10])) # tf.zeros # 784 x 10 행렬을 0으로 initialization 0에서 9까지 10 개필터
b = tf.Variable(tf.zeros([10]))     # 10 차원 벡터 0으로 initialization


sess.run(tf.global_variables_initializer())
y = tf.nn.softmax(tf.matmul(x,W) + b) # x.W 행렬을 곱한다음 b를 더함
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
#cross_entropy 는 정답과의 오차를 의미한다 
#reduce_sum  정답인 y_와 




In [3]:
#가중치 초기화 #
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  #initial = tf.get_variable("W_conv1",shape,initializer=tf.contrib.layers.xavier_initializer())
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

#합성곱(Convolution)과 풀링(Pooling)#############33#

def conv2d(x, W): # 
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# 

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

##########################################################

#첫 번째 레이어 #
#W_conv1 = weight_variable([5, 5, 1, 32]) #[x,y,입력채널, 출력채널(feature 를 32개로 뽑아냄 ) ]
W_conv1 = tf.get_variable("W_conv1",shape=[5,5,1,32],
                          initializer=tf.contrib.layers.xavier_initializer())
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


#두 번째 레이어#
#W_conv2 = weight_variable([5, 5, 32, 64]) [x,y, 입력채널 ,출력채널(feature 를 64개로 뽑아냄 )]
W_conv2 = tf.get_variable("W_conv2",shape=[5,5,32,64],
                          initializer=tf.contrib.layers.xavier_initializer())
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#세 번째 레이어#
#W_conv3 = weight_variable([5, 5, 64, 128]) [x,y 입력채널, 출력채널(feature 를 128개로 뽑아냄 )]
W_conv3 = tf.get_variable("W_conv3",shape=[5,5,64,128],
                          initializer=tf.contrib.layers.xavier_initializer())
# W(가중치) 를 랜덤하게 주기 위해서 xavier 를 사용함 
#그전에는 0.1 사용했음, weight 를 0으로 안주는 이유는 ReLu 때문에 다 사라짐 

b_conv3 = bias_variable([128])
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)


 

In [5]:

#완전 연결 계층 (Fully-Connected Layer)#
W_fc1 = weight_variable([4* 4 *128, 1024])
b_fc1 = bias_variable([1024])
h_pool3_flat = tf.reshape(h_pool3, [-1, 4*4*128])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)


#드롭아웃 (Dropout)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))



train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(10000):
  batch = mnist.train.next_batch(50) #각각의 훈련단계에서 50개의 훈련 샘플을 추출 
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={ 
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.08
step 100, training accuracy 0.84
step 200, training accuracy 0.98
step 300, training accuracy 0.94
step 400, training accuracy 0.94
step 500, training accuracy 0.94
step 600, training accuracy 0.96
step 700, training accuracy 0.98
step 800, training accuracy 1
step 900, training accuracy 1
step 1000, training accuracy 0.96
step 1100, training accuracy 0.98
step 1200, training accuracy 0.96
step 1300, training accuracy 1
step 1400, training accuracy 0.98
step 1500, training accuracy 0.94
step 1600, training accuracy 0.94
step 1700, training accuracy 0.94
step 1800, training accuracy 0.98
step 1900, training accuracy 1
step 2000, training accuracy 0.98
step 2100, training accuracy 0.98
step 2200, training accuracy 1
step 2300, training accuracy 0.98
step 2400, training accuracy 0.96
step 2500, training accuracy 1
step 2600, training accuracy 0.98
step 2700, training accuracy 0.98
step 2800, training accuracy 1
step 2900, training accuracy 0.98
step 3000, tr